<a href="https://colab.research.google.com/github/Shashank123-hub/MachineLearning/blob/master/Handwritten_Digits_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Project topic : Handwritten Digit Recognition**

> The goal of this topic is to recognize handwritten digits by the user using the Convolution Neural Networks(CNNs).  
We'll also build a GUI in which we can draw as well as recognize the digit right away.


> Modules used in this projects are the **Keras library** for deep learning and the **Tkinter library** for building the GUI.

> Also, the dataset used here is the **MNIST dataset**. It contains 60,000 images of handwritten digits from zero to nine and 10,000 images for testing. It has 10 different classes and the images are represented as 28*28 matrix with each cell contains with grayscale pixel value.





#**Step 1 : Installing the dependancies and load the dataset**

In [ ]:
!pip install numpy
!pip install tensorflow
!pip install keras
!pip install pillow

     |████████████████████████████████| 614kB 5.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 /usr/local/lib/python3.6/dist-packages/pip/_vendor/pep517/_in_process.py get_requires_for_build_wheel /tmp/tmp7p8m2ymd Check the logs for full command output.


In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

#Splitting the data between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step
(60000, 28, 28) (60000,)


#**Step 2 : Preprocess the data**

> This step primarily involves making our data ready for the neural network to be trained. The dimension of the training data is (60000,28,28) and the CNN model will require one more dimension to the shape (60000,28,28,1).    



In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes=None)
y_test = keras.utils.to_categorical(y_test, num_classes=None)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


print('x_train shape/Input dataset shape:', x_train.shape)
print('Train samples :',x_train.shape[0])
print('Test samples :',x_test.shape[0])

x_train shape/Input dataset shape: (60000, 28, 28, 1)
Train samples : 60000
Test samples : 10000


#**Step 3 : Creating the model**

> Our model simply consist of convolutional and pooling layer with dropouts for each layer.



> The optimizer used here is the **Adadelta optimizer**.



In [ ]:
num_classes = 10
epochs = 10
batch_size = 128

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape = input_shape)) #a convolutional neural layer
model.add(Conv2D(64, (3,3), activation='relu'))  #another CNN layer
model.add(MaxPooling2D(pool_size=(2,2)))  #pooling layer downsampling the images     
model.add(Dropout(0.25))  #dropout for the above layer
model.add(Flatten())  #flatten layer to convert data into 1-D/flattening the data
model.add(Dense(256, activation='relu'))  #third dense layer
model.add(Dropout(0.5))  #dropout for the above layer
model.add(Dense(num_classes, activation='softmax'))  #final dense layer  

model.compile(loss = keras.losses.categorical_crossentropy, 
optimizer = keras.optimizers.Adadelta() ,metrics=['accuracy'])


#**Step 4 : Training the compiled model**

In [ ]:
hist = model.fit(x_train, y_train, batch_size=batch_size,
epochs=epochs, verbose=1, validation_data=(x_test, y_test))

print("The model has been successfully trained")

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 160s 3ms/step - loss: 0.2201 - accuracy: 0.9311 - val_loss: 0.0571 - val_accuracy: 0.9810
Epoch 2/10
60000/60000 [==============================] - 161s 3ms/step - loss: 0.0712 - accuracy: 0.9782 - val_loss: 0.0439 - val_accuracy: 0.9850
Epoch 3/10
60000/60000 [==============================] - 160s 3ms/step - loss: 0.0522 - accuracy: 0.9842 - val_loss: 0.0330 - val_accuracy: 0.9895
Epoch 4/10
60000/60000 [==============================] - 160s 3ms/step - loss: 0.0413 - accuracy: 0.9868 - val_loss: 0.0280 - val_accuracy: 0.9906
Epoch 5/10
60000/60000 [==============================] - 159s 3ms/step - loss: 0.0339 - accuracy: 0.9893 - val_loss: 0.0276 - val_accuracy: 0.9906
Epoch 6/10
60000/60000 [==============================] - 160s 3ms/step - loss: 0.0293 - accuracy: 0.9909 - val_loss: 0.0273 - val_accuracy: 0.9914
Epoch 7/10
60000/60000 [==============================] - 159s

In [ ]:
model.save('HDRmodel.h5')
print('Saving the model as HDRmodel.h5')

Saving the model as HDRmodel.h5


#**Step 5 : Evaluating the model**

In [ ]:
score = model.evaluate(x_test, y_test, verbose =0)
print('Loss percentage = ', score[0]*100)
print('Accuracy percentage = ', score[1]*100)

Loss percentage =  2.5740840514458614
Accuracy percentage =  99.21000003814697


#**Step 6 : Creating GUI for prediction**

In [ ]:
!pip install win32
from keras.models import load_model
from tkinter import *
import tkinter as tk
#import win32gui
from PIL import ImageGrab, Image
import numpy as np

model = load_model('HDRmodel.h5')
def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)
    def clear_all(self):
        self.canvas.delete("all")
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
app = App()
mainloop()

ERROR: Could not find a version that satisfies the requirement win32 (from versions: none)
ERROR: No matching distribution found for win32


ImportError: ignored